**Sensor Independent Illumination Estimation for DNN Models**


This is a PyTorch implementation of the paper: "Sensor Independent Illumination Estimation for DNN Models." It is important to note that this implementation deviates from the original paper's code. For instance, it utilizes a lighter model architecture and employs different training parameters compared to those used in the original paper. Additionally, the code does not include the histogram learnable fall-off and scale factor introduced in the paper.

It is important to note that this code has not been thoroughly tested, and there is no guarantee that it will produce similar results to the paper. Furthermore, the code is licensed under the same license as the main GitHub page, which can be found at the following link: https://github.com/mahmoudnafifi/SIIE


Before you begin, please ensure to follow these instructions:


1. Create a training folder and upload the training images in PNG-16 format. Each image should have a corresponding .JSON file with the postfix '_metadata.json'. Inside the JSON file, include a key called "illuminant_color_raw" that specifies the ground-truth illuminant color for that particular image. It is **essential** to capture the training images using different sensors or cameras, as mentioned in the paper. This variation in sensor sources will assist the neural network in learning implicit sensor mappings.


2. Similarly, create a validation folder and upload the validation images using the same format and structure as described in step #1.

3. Create a testing folder and upload the testing images in the same format and structure as mentioned in step #1. It is optional to provide the ground-truth information for the testing images.

In case you encounter memory issues, consider implementing the following to mitigate the problem: 1) reduce the batch size, 2) decrease the input image size, and/or 3) trim model weights.


If you find this code useful, please cite the following paper:

Mahmoud Afifi and Michael S. Brown. "Sensor Independent Illumination Estimation
for DNN Models". In BMVC, 2019.




```
@inproceedings{afifi2019SIIE,
 title={Sensor-Independent Illumination Estimation for DNN Models},
 author={Afifi, Mahmoud and Brown, Michael S},
 booktitle={British Machine Vision Conference (BMVC)},
 pages={},
 year={2019}
}
```






In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import cv2
import os
from os.path import join
from os import listdir
import numpy as np
from glob import glob
import logging
import json
from torch import optim
from tqdm import tqdm
import math
import sys
import pickle

EPS = 1e-9
PI = 22.0 / 7.0

Input/output information

In [ ]:
trdir = 'drive/training'  # training dir
valdir = 'drive/validation'  # validation dir
testing_dir = 'drive/testing'  # testing dir
model_dir = 'drive/models/'  # directory to save final model
dir_checkpoint = 'drive/checkpoints/'  # directory to save checkpoints
result_dir = 'drive/result/'  # directory to save testing results
initial_est = False
epochs = 120  # number of epochs
batchsize = 16 # batch size
lr = 0.00001 # learning rate
l2reg = 0.0001 # L2 regularization factor
max_img_size = 120
validation_freq = 2  # validation frequency
chkpointperiod = 2  # checkpoint frequency

Histogram block

In [ ]:
"""
 If you find this histogram block useful, please cite our papers:

 Mahmoud Afifi and Michael S. Brown. "Sensor Independent Illumination Estimation
 for DNN Models". In BMVC, 2019.

 Mahmoud Afifi, Marcus A. Brubaker, and Michael S. Brown. "HistoGAN:
 Controlling Colors of GAN-Generated and Real Images via Color Histograms."
 In CVPR, 2021.

 @inproceedings{afifi2019SIIE,
  title={Sensor-Independent Illumination Estimation for DNN Models},
  author={Afifi, Mahmoud and Brown, Michael S},
  booktitle={British Machine Vision Conference (BMVC)},
  pages={},
  year={2019}
 }

 @inproceedings{afifi2021histogan,
  title={Histo{GAN}: Controlling Colors of {GAN}-Generated and Real Images via
  Color Histograms},
  author={Afifi, Mahmoud and Brubaker, Marcus A. and Brown, Michael S.},
  booktitle={CVPR},
  year={2021}
}
"""

class RGBuvHistBlock(nn.Module):
  def __init__(self, h=64, insz=max_img_size, resizing='interpolation',
               method='inverse-quadratic', sigma=0.02, intensity_scale=True,
               hist_boundary=None, green_only=False, device='cuda'):
    """ Computes the RGB-uv histogram feature of a given image.
    Args:
      h: histogram dimension size (scalar). The default value is 64.
      insz: maximum size of the input image; if it is larger than this size, the
        image will be resized (scalar).
      resizing: resizing method if applicable. Options are: 'interpolation' or
        'sampling'. Default is 'interpolation'.
      method: the method used to count the number of pixels for each bin in the
        histogram feature. Options are: 'thresholding', 'RBF' (radial basis
        function), or 'inverse-quadratic'. Default value is 'inverse-quadratic'.
      sigma: if the method value is 'RBF' or 'inverse-quadratic', then this is
        the learnable sigma parameter of the kernel function.
      intensity_scale: boolean variable to use the intensity scale (I_y in
        Equation 2). Default value is True.
      hist_boundary: a list of histogram boundary values.
      green_only: boolean variable to use only the log(g/r), log(g/b) channels.
        Default is False.

    Methods:
      forward: accepts input image and returns its histogram feature. Note that
        unless the method is 'thresholding', this is a differentiable function
        and can be easily integrated with the loss function. As mentioned in the
         paper, the 'inverse-quadratic' was found more stable than 'RBF' in our
         training.
    """
    super(RGBuvHistBlock, self).__init__()
    self.h = h
    self.insz = insz
    self.device = device
    self.resizing = resizing
    self.method = method
    self.intensity_scale = intensity_scale
    self.green_only = green_only
    if hist_boundary is None:
      hist_boundary = [-2.6, 2.6]
    hist_boundary.sort()
    self.hist_boundary = hist_boundary
    if self.method == 'thresholding':
      self.eps = (abs(hist_boundary[0]) + abs(hist_boundary [1])) / h
    else:
      self.sigma = sigma

  def forward(self, x):
    x = torch.clamp(x, 0, 1)
    if x.shape[2] > self.insz or x.shape[3] > self.insz:
      if self.resizing == 'interpolation':
        x_sampled = F.interpolate(x, size=(self.insz, self.insz),
                                  mode='bilinear', align_corners=False)
      elif self.resizing == 'sampling':
        inds_1 = torch.LongTensor(
          np.linspace(0, x.shape[2], self.h, endpoint=False)).to(
          device=self.device)
        inds_2 = torch.LongTensor(
          np.linspace(0, x.shape[3], self.h, endpoint=False)).to(
          device=self.device)
        x_sampled = x.index_select(2, inds_1)
        x_sampled = x_sampled.index_select(3, inds_2)
      else:
        raise Exception(
          f'Wrong resizing method. It should be: interpolation or sampling. '
          f'But the given value is {self.resizing}.')
    else:
      x_sampled = x

    L = x_sampled.shape[0]  # size of mini-batch
    if x_sampled.shape[1] > 3:
      x_sampled = x_sampled[:, :3, :, :]
    X = torch.unbind(x_sampled, dim=0)
    hists = torch.zeros((x_sampled.shape[0], 1 + int(not self.green_only) * 2,
                         self.h, self.h)).to(device=self.device)
    for l in range(L):
      I = torch.t(torch.reshape(X[l], (3, -1)))
      II = torch.pow(I, 2)
      if self.intensity_scale:
        Iy = torch.unsqueeze(torch.sqrt(II[:, 0] + II[:, 1] + II[:, 2] + EPS),
                             dim=1)
      else:
        Iy = 1
      if not self.green_only:
        Iu0 = torch.unsqueeze(torch.log(I[:, 0] + EPS) - torch.log(I[:, 1] +
                                                                   EPS), dim=1)
        Iv0 = torch.unsqueeze(torch.log(I[:, 0] + EPS) - torch.log(I[:, 2] +
                                                                   EPS), dim=1)
        diff_u0 = abs(
          Iu0 - torch.unsqueeze(torch.tensor(np.linspace(
              self.hist_boundary[0], self.hist_boundary[1], num=self.h)),
              dim=0).to(self.device))
        diff_v0 = abs(
          Iv0 - torch.unsqueeze(torch.tensor(np.linspace(
              self.hist_boundary[0], self.hist_boundary[1], num=self.h)),
              dim=0).to(self.device))
        if self.method == 'thresholding':
          diff_u0 = torch.reshape(diff_u0, (-1, self.h)) <= self.eps / 2
          diff_v0 = torch.reshape(diff_v0, (-1, self.h)) <= self.eps / 2
        elif self.method == 'RBF':
          diff_u0 = torch.pow(torch.reshape(diff_u0, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_v0 = torch.pow(torch.reshape(diff_v0, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_u0 = torch.exp(-diff_u0)  # Radial basis function
          diff_v0 = torch.exp(-diff_v0)
        elif self.method == 'inverse-quadratic':
          diff_u0 = torch.pow(torch.reshape(diff_u0, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_v0 = torch.pow(torch.reshape(diff_v0, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_u0 = 1 / (1 + diff_u0)  # Inverse quadratic
          diff_v0 = 1 / (1 + diff_v0)
        else:
          raise Exception(
            f'Wrong kernel method. It should be either thresholding, RBF,'
            f' inverse-quadratic. But the given value is {self.method}.')
        diff_u0 = diff_u0.type(torch.float32)
        diff_v0 = diff_v0.type(torch.float32)
        a = torch.t(Iy * diff_u0)
        hists[l, 0, :, :] = torch.mm(a, diff_v0)

      Iu1 = torch.unsqueeze(torch.log(I[:, 1] + EPS) - torch.log(I[:, 0] + EPS),
                            dim=1)
      Iv1 = torch.unsqueeze(torch.log(I[:, 1] + EPS) - torch.log(I[:, 2] + EPS),
                            dim=1)
      diff_u1 = abs(
        Iu1 - torch.unsqueeze(torch.tensor(np.linspace(
            self.hist_boundary[0], self.hist_boundary[1], num=self.h)),
            dim=0).to(self.device))
      diff_v1 = abs(
        Iv1 - torch.unsqueeze(torch.tensor(np.linspace(
            self.hist_boundary[0], self.hist_boundary[1], num=self.h)),
            dim=0).to(self.device))

      if self.method == 'thresholding':
        diff_u1 = torch.reshape(diff_u1, (-1, self.h)) <= self.eps / 2
        diff_v1 = torch.reshape(diff_v1, (-1, self.h)) <= self.eps / 2
      elif self.method == 'RBF':
        diff_u1 = torch.pow(torch.reshape(diff_u1, (-1, self.h)),
                            2) / self.sigma ** 2
        diff_v1 = torch.pow(torch.reshape(diff_v1, (-1, self.h)),
                            2) / self.sigma ** 2
        diff_u1 = torch.exp(-diff_u1)  # Gaussian
        diff_v1 = torch.exp(-diff_v1)
      elif self.method == 'inverse-quadratic':
        diff_u1 = torch.pow(torch.reshape(diff_u1, (-1, self.h)),
                            2) / self.sigma ** 2
        diff_v1 = torch.pow(torch.reshape(diff_v1, (-1, self.h)),
                            2) / self.sigma ** 2
        diff_u1 = 1 / (1 + diff_u1)  # Inverse quadratic
        diff_v1 = 1 / (1 + diff_v1)

      diff_u1 = diff_u1.type(torch.float32)
      diff_v1 = diff_v1.type(torch.float32)
      a = torch.t(Iy * diff_u1)
      if not self.green_only:
        hists[l, 1, :, :] = torch.mm(a, diff_v1)
      else:
        hists[l, 0, :, :] = torch.mm(a, diff_v1)

      if not self.green_only:
        Iu2 = torch.unsqueeze(torch.log(I[:, 2] + EPS) - torch.log(I[:, 0] +
                                                                   EPS), dim=1)
        Iv2 = torch.unsqueeze(torch.log(I[:, 2] + EPS) - torch.log(I[:, 1] +
                                                                   EPS), dim=1)
        diff_u2 = abs(
          Iu2 - torch.unsqueeze(torch.tensor(np.linspace(
              self.hist_boundary[0], self.hist_boundary[1], num=self.h)),
              dim=0).to(self.device))
        diff_v2 = abs(
          Iv2 - torch.unsqueeze(torch.tensor(np.linspace(
              self.hist_boundary[0], self.hist_boundary[1], num=self.h)),
              dim=0).to(self.device))
        if self.method == 'thresholding':
          diff_u2 = torch.reshape(diff_u2, (-1, self.h)) <= self.eps / 2
          diff_v2 = torch.reshape(diff_v2, (-1, self.h)) <= self.eps / 2
        elif self.method == 'RBF':
          diff_u2 = torch.pow(torch.reshape(diff_u2, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_v2 = torch.pow(torch.reshape(diff_v2, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_u2 = torch.exp(-diff_u2)  # Gaussian
          diff_v2 = torch.exp(-diff_v2)
        elif self.method == 'inverse-quadratic':
          diff_u2 = torch.pow(torch.reshape(diff_u2, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_v2 = torch.pow(torch.reshape(diff_v2, (-1, self.h)),
                              2) / self.sigma ** 2
          diff_u2 = 1 / (1 + diff_u2)  # Inverse quadratic
          diff_v2 = 1 / (1 + diff_v2)
        diff_u2 = diff_u2.type(torch.float32)
        diff_v2 = diff_v2.type(torch.float32)
        a = torch.t(Iy * diff_u2)
        hists[l, 2, :, :] = torch.mm(a, diff_v2)

    # normalization
    hists_normalized = hists / (
        ((hists.sum(dim=1)).sum(dim=1)).sum(dim=1).view(-1, 1, 1, 1) + EPS)

    return hists_normalized

Data loader

In [ ]:
from types import prepare_class
class BasicDataset(Dataset):
    def __init__(self, imgs_dir):
        self.imgs_dir = imgs_dir
        self.imgfiles = [join(imgs_dir, file) for file in listdir(imgs_dir)
                        if file.endswith('.png') and not file.startswith('.')]
        logging.info(f'Creating dataset with {len(self.imgfiles)} examples')

    def __len__(self):
        return len(self.imgfiles)

    @classmethod
    def preprocess(cls, img):
        img = cv2.resize(img, (max_img_size, max_img_size))
        # HWC to CHW
        img_chw = img.transpose((2, 0, 1))
        return img_chw

    def __getitem__(self, i):
        gt_ext = ('_metadata.json')
        img_file = self.imgfiles[i]
        gt_file = img_file.replace('.png', gt_ext)
        img = cv2.imread(img_file, -1) / ((2 ** 16) - 1)
        img = img[..., ::-1]
        img = self.preprocess(img)
        with open(gt_file, 'r') as f:
          data = json.load(f)
          gt_ill = np.array(data['illuminant_color_raw'])
        return {'image': torch.from_numpy(img.copy()),
                'gt_ill': torch.from_numpy(gt_ill)}

Loss function

In [ ]:
def angular_loss(predicted, gt, shrink=True):
  """Computes angular error between predicted and gt illuminant color(s)."""
  cossim = torch.clamp(torch.sum(predicted * gt, dim=1) / (
      torch.norm(predicted, dim=1) * torch.norm(gt, dim=1) + EPS), -1, 1.)
  if shrink:
    angle = torch.acos(cossim * 0.999999)
  else:
    angle = torch.acos(cossim)
  a_error = 180.0 / PI * angle
  a_error = torch.sum(a_error) / a_error.shape[0]
  return a_error

Network

In [ ]:
class Flatten(nn.Module):
    def forward(self, x):
        x = x.view(x.size()[0], -1)
        return x

class Net(nn.Module):
    def __init__(self, output_chs=3, device='cude'):
      """Network (sensor mapping / illuminant estimation) constructor."""
      super(Net, self).__init__()
      self._net = nn.Sequential(
          nn.Conv2d(3, 92, kernel_size=5, padding=0, stride=2),
          nn.ReLU(inplace=True),
          nn.Conv2d(92, 128, kernel_size=3, padding=0, stride=2),
          nn.ReLU(inplace=True),
          nn.Conv2d(128, 256, kernel_size=2, padding=0, stride=1),
          nn.ReLU(inplace=True),
          Flatten(),
          torch.nn.Linear(13*13*256, output_chs, bias=True))
      self._hist_block = RGBuvHistBlock(insz=max_img_size, h=61)

    def forward(self, img):
      """Forward function."""
      return self._net(self._hist_block(img))


class SIIE(nn.Module):
  def __init__(self, device='cuda', initial_est=True):
    """SIIE net constructor."""
    super(SIIE, self).__init__()
    self.device = device
    self._initial_est = initial_est

    # Sensor mapping net
    self._sensor_mapping_net = Net(output_chs=9)
    self._sensor_mapping_net.to(device=device)

    # Illuminant estimation net
    self._illum_est_net = Net(output_chs=3)

  def forward(self, img):
    """Forward function of SIIE net."""
    if self._initial_est:
      initial_ill = torch.mean(img, dim=[2, 3])
      gains = torch.unsqueeze(initial_ill[:, 1], dim=-1) / initial_ill
      img[:, 0, ...] *= torch.unsqueeze(torch.unsqueeze(gains[:, 0], axis=-1),
                                        axis=-1)
      img[:, 2, ...] *= torch.unsqueeze(torch.unsqueeze(gains[:, 2], axis=-1),
                                        axis=-1)

    m = self._sensor_mapping_net(img)
    m = m.reshape([-1, 3, 3])
    m = torch.abs(m) / (
        torch.unsqueeze(torch.sum(torch.abs(m), axis=-1), axis=-1) + EPS)
    img_mapped = img.clone()
    for i in range(m.shape[0]):
      temp = torch.mm(torch.squeeze(m[i, :, :]),
                      torch.reshape(torch.squeeze(
                          img[i, :, :, :]), (3, -1)))
      img_mapped[i, :, :, :] = torch.reshape(temp, (img.shape[1], img.shape[2],
                                                    img.shape[3]))

    ill_mapped = self._illum_est_net(img_mapped)
    inds = (torch.linalg.matrix_rank(m) < 3).to(device=m.device)
    offset = torch.zeros(m.shape[0], 1).to(device=m.device)
    if torch.sum(inds) > 0:
      offset[inds, ...] = torch.unsqueeze(
          torch.rand(torch.sum(inds)), axis=-1).to(device=m.device) * 0.0001
    inv_m = torch.linalg.inv(m + torch.unsqueeze(offset, axis=-1))
    ill = ill_mapped.clone()
    for i in range(inv_m.shape[0]):
      temp = torch.mm(torch.squeeze(inv_m[i, :, :]),
                      torch.unsqueeze(ill_mapped[i, :], axis=-1))
      ill[i, :] = torch.reshape(temp, ill[i, :].shape)
    if self._initial_est:
      ill = initial_ill * ill
    return ill, m

Training

In [ ]:
def train_net(net, device, train_dir, val_dir, epochs=300,
              batch_size=8, lr=0.00001, l2reg=0.0001,
              chkpointperiod=20, save_cp=True):
    retain_graph=True
    train = BasicDataset(train_dir)
    val = BasicDataset(val_dir)
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True,
                              num_workers=2, pin_memory=True)
    val_loader = DataLoader(val, batch_size=batch_size, shuffle=False,
                            num_workers=2, pin_memory=True, drop_last=True)
    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs} epochs
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {len(train)}
        Validation size: {len(val)}
        Checkpoints:     {save_cp}
        Device:          {device.type}
    ''')

    optimizer = optim.Adam(net.parameters(), lr=lr, betas=(0.85, 0.999),
                           eps=1e-08, weight_decay=l2reg)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

    for epoch in range(epochs):
        net.train()

        epoch_loss = 0
        with tqdm(total=len(train), desc=f'Epoch {epoch + 1}/{epochs}',
                  unit='img') as pbar:
            for batch in train_loader:
                imgs = batch['image']
                gt_ill = batch['gt_ill']
                imgs = imgs.to(device=device, dtype=torch.float32)
                gt_ill = gt_ill.to(device=device, dtype=torch.float32)
                est_ill, _ = net(imgs)
                loss = angular_loss(est_ill, gt_ill)
                epoch_loss += loss.item()
                pbar.set_postfix(**{'loss (batch)': loss.item()})
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                pbar.update(np.ceil(imgs.shape[0]))
                global_step += 1
        if (epoch + 1) % validation_freq == 0:
          val_score = vald_net(net, val_loader, device)
          print(f'~~~~ Validation loss: {val_score} ~~~~~')

        if save_cp and (epoch + 1) % chkpointperiod == 0:
            if not os.path.exists(dir_checkpoint):
                os.mkdir(dir_checkpoint)
                logging.info('Created checkpoint directory')

            torch.save(net.state_dict(), dir_checkpoint +
                       f'siie_net_{epoch + 1}.pth')
            logging.info(f'Checkpoint {epoch + 1} saved!')
        scheduler.step()

    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
        logging.info('Created trained models directory')
    torch.save(net.state_dict(), model_dir + 'siie_net.pth')
    logging.info('Saved trained model!')
    logging.info('End of training')


def vald_net(net, loader, device):
    """Evaluation using MAE"""
    net.eval()
    n_val = len(loader) + 1
    mae = 0

    with tqdm(total=n_val, desc='Validation round', unit='batch',
              leave=False) as pbar:
        for batch in loader:
            imgs = batch['image']
            gt_ill = batch['gt_ill']
            imgs = imgs.to(device=device, dtype=torch.float32)
            gt_ill = gt_ill.to(device=device, dtype=torch.float32)

            with torch.no_grad():
                est_ill, m = net(imgs)
                loss = angular_loss(gt_ill, est_ill)
                mae = mae + loss

            pbar.update(np.ceil(imgs.shape[0]))
    net.train()
    return mae / n_val


logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logging.info('Training of SIIE Net')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
net = SIIE(device=device, initial_est=initial_est)
net.to(device=device)

try:
  train_net(net=net, device=device, train_dir=trdir,
            val_dir=valdir, epochs=epochs, l2reg=l2reg,
            batch_size=batchsize, lr=lr, chkpointperiod=chkpointperiod)
except KeyboardInterrupt:
    torch.save(net.state_dict(), 'intrrupted_check_point.pth')
    logging.info('Saved interrupt checkpoint backup')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)

Testing

In [ ]:
def test_net(net, device, test_dir):
    gt_ext = ('_metadata.json')
    test = BasicDataset(test_dir)

    imgfiles = [join(test_dir, file) for file in listdir(test_dir)
                if file.endswith('.png') and not file.startswith('.')]
    logging.info(f'Testing {len(imgfiles)} images')
    ae = np.ones((len(imgfiles), 1)) * -1
    estimated_ills = np.zeros((len(imgfiles), 3))
    gt_ills = np.zeros((len(imgfiles), 3))
    filenames = []
    for i, f in enumerate(imgfiles):
      filenames.append(f)
      gt_file = f.replace('.png', gt_ext)
      img = cv2.imread(f, -1) / ((2 ** 16) - 1)
      img = img[..., ::-1]
      if os.path.exists(gt_file):
        with open(gt_file, 'r') as f:
          data = json.load(f)
          gt_ill = np.array(data['illuminant_color_raw'])
          gt_ill = torch.from_numpy(gt_ill)
      else:
        gt_ill = None
      img = cv2.resize(img, (max_img_size, max_img_size))
      # HWC to CHW
      img_chw = img.transpose((2, 0, 1))
      in_img = torch.from_numpy(img_chw.copy())
      in_img = in_img.to(device=device, dtype=torch.float32)
      in_img = torch.unsqueeze(in_img, axis=0)
      est_ill, _ = net(in_img)
      if gt_ill is not None:
        gt_ill = gt_ill.to(device=device, dtype=torch.float32)
        gt_ill = torch.unsqueeze(gt_ill, axis=0)
        loss = angular_loss(gt_ill, est_ill, shrink=False)
        ae[i] = loss.detach().cpu().numpy()
        gt = gt_ill.detach().cpu().numpy()
        gt = gt / np.sqrt(np.sum(gt ** 2))
        gt_ills[i, :] = gt.reshape(1, 3)
      result = est_ill.detach().cpu().numpy()
      result = result / np.sqrt(np.sum(result ** 2))
      estimated_ills[i, :] = result
    return estimated_ills, ae, filenames, gt_ills

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device {device}')
net = SIIE(device=device, initial_est=initial_est)
net.to(device=device)
net.load_state_dict(torch.load(os.path.join(model_dir, 'siie_net.pth'),
                                               map_location=device))
estimated_illuminants, ae, filenames, gt_ill = test_net(net, device,
                                                        testing_dir)
data = {'filenames': filenames,
        'estiamted_ills': estimated_illuminants.tolist(),
        'gt_ills': gt_ill.tolist(), 'ae': ae.tolist()}
json_object = json.dumps(data, indent=4)
# Writing to sample.json
if not os.path.exists(result_dir):
  os.mkdir(result_dir)
with open(os.path.join(result_dir, "results.json"), "w") as outfile:
    outfile.write(json_object)
